<a href="https://colab.research.google.com/github/Aminfaraji/ABINetpaddleocr/blob/main/salhe_job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import numpy as np
!pip install torchmetrics
!sudo apt-get install tesseract-ocr-fas
!pip install pytesseract
from torchmetrics.text import CharErrorRate ,WordErrorRate
import pytesseract
from glob import glob
import shutil,os,cv2
from difflib import SequenceMatcher
from PIL import Image
from tqdm import tqdm
!pip install easyocr
import easyocr
from concurrent.futures import ThreadPoolExecutor, as_completed

In [6]:
# # from IPython.display import Image
# from PIL import Image
# import requests

# def show_image(image_name):
#   return Image(filename=image_name)
# reader_en_fr = easyocr.Reader(['en', 'fa'])

# def read_text(image_name, model_name, in_line=False):

#   # Read the data
#   image_name = Image.open(image_name)
#   text = model_name.readtext(image_name, detail = 0, paragraph=in_line)

#   return '\n'.join(text)

In [ ]:
# # show_image("/content/drive/MyDrive/images/001A10.jpg")
# fr_text = read_text("/content/drive/MyDrive/images/001A10.jpg", reader_en_fr)
# print(fr_text)

In [ ]:
import xml.etree.ElementTree as ET
from PIL import Image
import pytesseract
import pandas as pd
import os

def extract_texts_with_bndbox(element):
    results = []
    for obj in element.findall('.//object'):
        content_element = obj.find('content')
        bndbox_element = obj.find('bndbox')
        if content_element is not None and bndbox_element is not None:
            text = content_element.text.strip()
            bndbox = {
                'xmin': int(bndbox_element.find('xmin').text),
                'ymin': int(bndbox_element.find('ymin').text),
                'xmax': int(bndbox_element.find('xmax').text),
                'ymax': int(bndbox_element.find('ymax').text),
            }
            results.append((text, bndbox))
    return results

def crop_image(image_path, bndbox):
  image = cv2.imread(image_path)
  X = bndbox['xmin']
  Y = bndbox['ymin']
  W = bndbox['xmax']
  H = bndbox['ymax']
  cropped_image = image[Y:H, X:W]
  return cropped_image

def ocr_image_tesseract(image):
    text = pytesseract.image_to_string(image, lang='eng+fa')
    return text

def ocr_image_easyocr(reader, image):
  result = reader.readtext(image , paragraph=True)
  text = ' '.join([res[1] for res in result])
  return text

def calculate_error_rates(ground_truth, ocr_text):
    char_error_rate = cer(ocr_text , ground_truth)
    word_error_rate = wer(ocr_text , ground_truth)
    return char_error_rate, word_error_rate

def process_xml_file(xml_path, image_path, reader):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    texts_with_bndbox = extract_texts_with_bndbox(root)

    results = []
    for text, bndbox in texts_with_bndbox:
        cropped_image = crop_image(image_path, bndbox)

        ocr_text_tesseract = ocr_image_tesseract(cropped_image)
        ocr_text_easyocr = ocr_image_easyocr(reader, cropped_image)

        char_error_rate_tesseract, word_error_rate_tesseract = calculate_error_rates(text, ocr_text_tesseract)
        char_error_rate_easyocr, word_error_rate_easyocr = calculate_error_rates(text, ocr_text_easyocr)

        results.append({
            'Original Text': text,
            'OCR Text Tesseract': ocr_text_tesseract,
            'OCR Text EasyOCR': ocr_text_easyocr,
            'Bounding Box': f"{bndbox['xmin']},{bndbox['ymin']},{bndbox['xmax']},{bndbox['ymax']}",
            'Character Error Rate Tesseract': char_error_rate_tesseract.item(),
            'Word Error Rate Tesseract': word_error_rate_tesseract.item(),
            'Character Error Rate EasyOCR': char_error_rate_easyocr.item(),
            'Word Error Rate EasyOCR': word_error_rate_easyocr.item(),
            'Image Path': os.path.basename(image_path),
            'XML File': os.path.basename(xml_path)
        })
    return results

xml_files =glob('/content/drive/MyDrive/ArnaDataset/*.xml')
image_files = '/content/drive/MyDrive/images/'
all_data = []
reader = easyocr.Reader(['en' , 'fa'])


cer = CharErrorRate()
wer = WordErrorRate()

with ThreadPoolExecutor(max_workers=8) as executor:

    futures = []
    with tqdm(total=len(xml_files)) as pbar:
      for xml in xml_files:
          futures.append(executor.submit(process_xml_file, xml, image_files+os.path.split(xml)[1][:-4]+'.jpg' , reader))

      for future in as_completed(futures):
          all_data.extend(future.result())
          pbar.update(1)

# for xml in tqdm(xml_files):
#   all_data.extend(process_xml_file(xml,image_files+os.path.split(xml)[1][:-4]+'.jpg' , reader))

df = pd.DataFrame(all_data)
df.to_excel('output.xlsx', index=False)

print("Data has been saved to output.xlsx")

  0%|          | 0/1001 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:911: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:1424.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,
  1%|          | 7/1001 [01:44<3:27:26, 12.52s/it]/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric WordErrorRate was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
  1%|          | 10/1001 [02:25<3:30:16, 12.73s/it]/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of me

In [1]:
# df = pd.DataFrame(all_data)
# df

In [38]:
bndbox = {'xmin' : 725 , 'ymin':124 , 'xmax':1116 , 'ymax' : 176}
cropped_image = crop_image(image_files+'/011C9.jpg', bndbox)
def ocr_image_easyocr(reader, image):
  # image = cv2.imread(image)
  result = reader.readtext(image , paragraph=True)
  text = ' '.join([res[1] for res in result])
  return text

In [ ]:
cropped_image

In [ ]:
ocr_image_easyocr(reader ,cropped_image)

In [56]:
bndbox = {'xmin' : 725 , 'ymin':124 , 'xmax':1116 , 'ymax' : 176}

def crop_image(image_path, bndbox):
  image = cv2.imread(image_path)
  X = bndbox['xmin']
  Y = bndbox['ymin']
  W = bndbox['xmax']
  H = bndbox['ymax']
  cropped_image = image[Y:H, X:W]
  return cropped_image

In [58]:
cropped_image = crop_image(image_files+'/011C9.jpg', bndbox)
ocr_image_easyocr(reader , cropped_image)

'شرکت های زیر مجموعه'